## Renames Viewpoint Retrieval Eyetracking Files (.sh)

VERSION: `Rename_Viewpoint_Retrieval_Sourcedata`

In [ ]:
#!/bin/bash

renamed_folders_count=0

extract_date() {
    local file="$1"
    local date_info=$(strings "$file" | grep "^DATE:" | awk -F "DATE:" '{print $2}')
    local month=$(echo "$date_info" | awk '{print $2}')
    local day=$(echo "$date_info" | awk '{print $3}')
    local year=$(echo "$date_info" | awk '{print $NF}')

    local month_num=$(date -d "$month 1 2000" +%m 2> /dev/null | sed 's/^0*//')

    if [[ -z $month_num ]]; then
        echo "Error: Unable to extract date from $file."
        return 1
    else
        printf -v date_str "%4d%02d%02d" "$year" "$month_num" "$day"
        echo "$date_str"
    fi
}

rename_file() {
    local file="$1"
    local session_dir="$2"
    local id="$3"
    local date="$4"
    local extension="${file##*.}"
    local new_name="${session_dir}/sub-${id}_date-${date}_proj-viewpoint_task-recognition_eyetrack.${extension}"

    if [[ -n $new_name ]]; then
        mv "$file" "$new_name"
    fi
}

for id_folder in */; do
    if [[ -d "$id_folder" ]]; then
        id=${id_folder%/}
        for session_folder in "${id_folder}"*/; do
            session_name=$(basename "$session_folder")
            date=""

            while IFS= read -r -d '' file; do
                if [[ -z $date ]]; then
                    extracted_date=$(extract_date "$file")
                    if [[ $? -eq 0 ]]; then
                        date=$extracted_date
                    fi
                fi
                rename_file "$file" "$session_folder" "$id" "$date"
            done < <(find "$session_folder" -maxdepth 1 -type f \( -name "*.edf" -o -name "*.dat" -o -name "*.log" \) -print0)
            
            if [[ -z $date ]]; then
                echo "Error: Cannot find date for files in directory '$session_folder'"
                read -p "Enter the date for files in '$session_folder' in YYYYMMDD format (e.g. 20231211): " date
                while ! [[ $date =~ ^[0-9]{8}$ ]]; do
                    read -p "Invalid date format. Please enter a valid date for '$session_folder': " date
                done
            fi

            new_session_folder="${id_folder}${date}"
            if [[ ! -d "$new_session_folder" ]]; then
                mv "$session_folder" "$new_session_folder" && ((renamed_folders_count++))
                echo "$session_folder -> $new_session_folder"
            fi
        done
    fi
done

echo "Renaming completed."
echo "Total session folders renamed: $renamed_folders_count"
